NLP Assignment 2
Name: Chinmay Singhania
Roll No: BECOC337

In [68]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import logging
logging.basicConfig(format = "%(levelname)s - %(asctime)s: %(message)s",datefmt = '%H:%M:%S', level=logging.INFO)

In [69]:
df = pd.read_csv('CarDataset.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


Data Preprocessing

In [70]:
#New column for combined make and model is created
df['Maker_Model'] = df['Make']+" "+df['Model']

#All the columns containing text are chosen and put in a new dataframe df1
df1 = df[['Engine Fuel Type','Transmission Type','Driven_Wheels','Market Category','Vehicle Size','Vehicle Style', 'Maker_Model']]

#All the columns combined into one column in df2 dataframe
df2 = df1.apply(lambda x: ','.join(x.astype(str)),axis = 1)

#A new pandas dataframe is created of name df_clean containing column clean
df_clean = pd.DataFrame({'clean':df2}) 
df_clean.head()

,clean
0,"premium unleaded (required),MANUAL,rear wheel ..."
1,"premium unleaded (required),MANUAL,rear wheel ..."
2,"premium unleaded (required),MANUAL,rear wheel ..."
3,"premium unleaded (required),MANUAL,rear wheel ..."
4,"premium unleaded (required),MANUAL,rear wheel ..."


In [71]:
#List of list data corpus for Gensim modelling
sent = [row.split(',') for row in df_clean['clean']]
sent[:2]

[['premium unleaded (required)',
  'MANUAL',
  'rear wheel drive',
  'Factory Tuner',
  'Luxury',
  'High-Performance',
  'Compact',
  'Coupe',
  'BMW 1 Series M'],
 ['premium unleaded (required)',
  'MANUAL',
  'rear wheel drive',
  'Luxury',
  'Performance',
  'Compact',
  'Convertible',
  'BMW 1 Series']]

Counting Occurences

In [72]:
cv = CountVectorizer()
cv.fit(df2)
model = CountVectorizer(ngram_range=(1,1))
matrix = model.fit_transform(df2).toarray()
count_occurs_df = pd.DataFrame(data=matrix, columns=model.get_feature_names_out())
count_occurs_df.head()

,10,100,124,12c,15,150,1500,1500hd,16,190,...,xv,yaris,yorker,yukon,z3,z4,z71,z8,zdx,zephyr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Normalized Count Occurrences

In [73]:
norm_cv = TfidfVectorizer(use_idf=False, norm='l2')
norm_count_occurs = norm_cv.fit_transform(df2)
norm_count_occurs_df = pd.DataFrame(
    (count, word) for word, count in zip(
        norm_count_occurs.toarray().tolist()[0],
        norm_cv.get_feature_names_out()
    )
)
norm_count_occurs_df.columns = ['Word', 'Count']
norm_count_occurs_df.sort_values(
    'Count', ascending=False, inplace=True
)
norm_count_occurs_df.head()

,Word,Count
395,high,0.25
261,drive,0.25
809,wheel,0.25
563,premium,0.25
607,rear,0.25


Bag of Words

In [74]:
tfidf_vec = TfidfVectorizer()
tfidf_vec.fit(df2)
print(sorted(tfidf_vec.vocabulary_))
vector = tfidf_vec.transform(df2)

['10', '100', '124', '12c', '15', '150', '1500', '1500hd', '16', '190', '200', '200h', '200sx', '200t', '240', '240sx', '250', '2500', '250h', '2dr', '2x', '300', '3000gt', '300h', '300m', '300zx', '323', '330', '350', '350z', '360', '370z', '400', '400h', '420', '430', '450', '450h', '456m', '458', '460', '470', '4c', '4dr', '4runner', '4x', '50', '500', '500e', '500l', '500x', '550', '560', '57', '570', '570s', '575m', '599', '600', '6000', '600h', '612', '62', '626', '650s', '718', '740', '760', '780', '7x', '80', '850', '86', '90', '900', '9000', '911', '928', '929', '940', '944', '960', '968', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'acadia', 'accent', 'acclaim', 'accord', 'achieva', 'activehybrid', 'acura', 'aerio', 'aerostar', 'alero', 'alfa', 'all', 'allante', 'allroad', 'alltrack', 'alpina', 'altima', 'am', 'amanti', 'amg', 'and', 'armada', 'arnage', 'aspen', 'aspire', 'aston', 'astro', 'ats', 'audi', 'aurora', 'automated_manual', 'automatic', 'avalanche', 'avalon', 'avenger', 'av

Word2Vec

In [75]:
model = Word2Vec(sent, min_count=1, vector_size=50, workers=3, window=3, sg=1)

In [76]:
model.wv.get_vector("Toyota Camry")

array([-0.02467795,  0.12949774,  0.01945003, -0.07404717, -0.07622898,
       -0.22535115,  0.01007154,  0.2750467 , -0.12015145, -0.07350729,
        0.08122371,  0.03654144,  0.06054086, -0.0279253 , -0.03835192,
        0.18041871,  0.14997883,  0.29332575, -0.12456731, -0.33144012,
       -0.02205079, -0.0496365 ,  0.23544556,  0.07241936,  0.21748145,
       -0.01404196, -0.0541056 ,  0.40538234, -0.05844185, -0.02621067,
        0.04348486,  0.04836166,  0.06057474,  0.01080558,  0.06499628,
       -0.10815413,  0.21306634,  0.01690482,  0.06297053,  0.05057763,
        0.09133527, -0.04431102, -0.2250042 ,  0.07447414,  0.3622559 ,
        0.0809232 , -0.01289125, -0.15980566,  0.03139426,  0.05939862],
      dtype=float32)

Compare Similarity

In [77]:
model.wv.similarity("Porsche 718 Cayman", "Nissan Van")

0.80355424

In [78]:
model.wv.most_similar("Mercedes-Benz SLK-Class")

[('Lamborghini Murcielago', 0.9862064719200134),
 ('Ferrari 575M', 0.9844840168952942),
 ('Ferrari 456M', 0.9830684065818787),
 ('Tesla Model S', 0.9819356203079224),
 ('BMW 1 Series', 0.9811258316040039),
 ('Mercedes-Benz SL-Class', 0.9808278679847717),
 ('Ferrari 599', 0.9805040955543518),
 ('BMW Z4', 0.9796039462089539),
 ('Nissan GT-R', 0.9783667922019958),
 ('Ford Shelby GT500', 0.9780237674713135)]